In [1]:
import numpy as np
import pandas as pd
import pandas.io.data as web
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
pd.options.display.max_rows = 99999
pd.options.display.max_columns = 99999

In [2]:
df_SP500 = web.DataReader('^GSPC',data_source='yahoo',start='1/1/2000',end='1/1/2017')

In [3]:
df_SP500.head()

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,931800000,1455.219971
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1009000000,1399.420044
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1085500000,1402.109985
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1092300000,1403.449951
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1225200000,1441.469971


In [ ]:
df_SP500['Close'].plot(figsize=(20,10), grid=True)

### We want to look at the 2 month (i.e. 42 day) and one year (i.e. 252 day) rolling averages

In [ ]:
df_SP500['Close: 42d RA'] = pd.rolling_mean(df_SP500['Close'], window=42)
df_SP500['Close: 252d RA'] = pd.rolling_mean(df_SP500['Close'], window=252)

In [ ]:
df_SP500[['Close','Close: 42d RA','Close: 252d RA']].plot(figsize = (30, 15))

# Trading Signals

### 0) Define some Signal Threshold
- Let SD = 50

### 1) Buy Signal (Go Long)

- 42-day mean for the first time goes SD points **above** the 252-day mean

### 2) Wait (Park in Cash)

- 42-day mean comes **within** +/- SD points of the 252-day mean

### 3) Sell Signal (Go Short)

- 42-day mean for the first time goes SD points **below** the 252-day mean

In [ ]:
df_SP500['Diff'] = df_SP500['Close: 42d RA'] - df_SP500['Close: 252d RA']

In [ ]:
SD = 50

df_SP500['Regime'] = (df_SP500['Diff']>SD).astype(int) - (df_SP500['Diff']<-SD).astype(int)

In [ ]:
df_SP500['Regime'].value_counts()

In [ ]:
df_SP500[pd.isnull(df_SP500['Regime'])==False]['Regime'].plot(figsize = (20,10), ylim=[-1.5, 1.5])

In [ ]:
df = df_SP500[pd.isnull(df_SP500['Diff'])==False]

fig, ax1 = plt.subplots()
t = df.index
s1 = df['Close']
s2 = df['Close: 42d RA']
s3 = df['Close: 252d RA']
ax1.plot(t, s1, 'b-')
ax1.plot(t, s2, 'r-')
ax1.plot(t, s3, 'y-')
ax1.set_xlabel('Date')
# Make the y-axis label and tick labels match the line color.
ax1.set_ylabel('Index', color='b')
for tl in ax1.get_yticklabels():
    tl.set_color('b')


ax2 = ax1.twinx()
ax2.set_ylim([-1.5,1.5])
s4 = df['Regime']
ax2.plot(t, s4, 'c--')
ax2.set_ylabel('Signal', color='c')
#import matplotlib.transforms as mtransforms
#trans = mtransforms.blended_transform_factory(ax2.transData, ax2.transAxes)
ax2.fill_between(t, 0, 1, where=s4 > 0, facecolor='green', alpha=0.5)
ax2.fill_between(t, -1, 0, where=s4 < 0, facecolor='red', alpha=0.5)
for tl in ax2.get_yticklabels():
    tl.set_color('c')
fig.set_size_inches(18.5, 10.5, forward=True)
plt.grid(False)
plt.show()

In [ ]:
df['Market log-Return'] = np.log(df['Close']/df['Close'].shift(1))

### Note: using arithmetic returns:

# $(1+r_{total}) = (1+r_{1})\cdot(1+r_{2})...\cdot(1+r_{N})$

### However, using log-returns:

# $log(1+r_{total}) = log\left ( \frac{P_{N}}{P_{0}} \right ) = log\left ( \frac{P_{N}}{P_{N-1}}\cdot\frac{P_{N-1}}{P_{N-2}}\cdot...\cdot\frac{P_{1}}{P_{0}} \right )$
# $= log(1+r_{N-1}) + log(1+r_{N-2}) +...+ log(1+r_{1})$


# i.e. take the sum of returns (not the product)

**Recall**: 
- The Regime columns with a value of 1 implies we are holding the index
    - So we book $1\cdot$(Market Return)
- The Regime columns with a value of 0 implies we're not in the market
    - So we book $0\cdot$(Market Return)
- The Regime columns with a value of -1 implies we've shorted the index
     - So we book $-1\cdot$(Market Return)
     
# i.e. just take the product df['Market log-Return]$*$df['Regime'] and take the cumsum

In [ ]:
df['Market log-Return']

In [ ]:
df['Strategy'] = df['Market log-Return']*df['Regime']

In [ ]:
df[['Market log-Return', 'Strategy']].cumsum().apply(np.exp).plot(figsize=(30,15))

In [ ]:
df2 = df[['Close', 'Close: 42d RA', 'Close: 252d RA','Diff', 'Market log-Return']].copy()

In [ ]:
for threshold in [10, 20, 30, 40, 50]:
    df2['Regime: '+str(threshold)] = (df2['Diff']>threshold).astype(int) - (df2['Diff']<-threshold).astype(int)

In [ ]:
#assume transaction cost = 10

transaction_cost = 10

for threshold in [10, 20, 30, 40, 50]:
    df2['Transaction Cost: '+str(threshold)] = (-transaction_cost)*(df2['Regime: '+str(threshold)]!=df2['Regime: '+str(threshold)].shift(-1)).astype(int)
    df2['Value: '+str(threshold)] = df2['Close']+df2['Transaction Cost: '+str(threshold)]

In [ ]:
for threshold in [10, 20, 30, 40, 50]:
    df2['log-Return: '+str(threshold)] = np.log(df2['Value: '+str(threshold)]/df2['Value: '+str(threshold)].shift(1))

In [ ]:
for threshold in [10, 20, 30, 40, 50]:
    df2['Strategy: '+str(threshold)] = df2['log-Return: '+str(threshold)]*df2['Regime: '+str(threshold)]

In [ ]:
df2[['Market log-Return', 'Strategy: 10', 'Strategy: 20', 'Strategy: 30', 'Strategy: 40', 'Strategy: 50']].cumsum().apply(np.exp).plot(figsize=(30,15))

In [ ]:
df3 = df[['Close', 'Close: 42d RA', 'Close: 252d RA','Diff', 'Market log-Return']].copy()

for transaction_cost in [40]:#[10,20,30,40,50]:
    for threshold in [10, 20, 30, 40, 50]:
        df3['Regime: '+str(threshold)] = (df3['Diff']>threshold).astype(int) - (df3['Diff']<-threshold).astype(int)
        df3['Transaction: '+str(threshold)] = (df3['Regime: '+str(threshold)]!=df3['Regime: '+str(threshold)].shift(-1)).astype(int)
        df3['Cumulative Transaction Costs: '+str(threshold)] = (transaction_cost)*df3['Transaction: '+str(threshold)].cumsum()
        df3['Value: '+str(threshold)] = df3['Close']-df3['Cumulative Transaction Costs: '+str(threshold)]
        df3['log-Return: '+str(threshold)] = np.log(df3['Value: '+str(threshold)]/df3['Value: '+str(threshold)].shift(1))
        df3['Strategy: '+str(threshold)] = df3['log-Return: '+str(threshold)]*df3['Regime: '+str(threshold)]
    df3[['Market log-Return', 'Strategy: 10', 'Strategy: 20', 'Strategy: 30', 'Strategy: 40', 'Strategy: 50']].cumsum().apply(np.exp).plot(figsize=(30,15), title = 'Transaction Cost: '+str(transaction_cost))

In [ ]:
df3.head()